# M7 New API Features
This notebook shows stable column ordering and sanitized headers via the orchestrator helpers, matching the CLI.

In [ ]:
from spicelab.analysis import GainBandwidthSpec, GainMarginSpec, PhaseMarginSpec, run_and_measure
from spicelab.core.circuit import Circuit
from spicelab.core.components import VA, C, Resistor
from spicelab.core.net import GND, Net
from spicelab.core.types import AnalysisSpec, SweepSpec
from spicelab.orchestrator import Job


def build_rc():
    c = Circuit("rc_ac")
    n_in, n_out = Net("vin"), Net("out")
    vin = VA(ac_mag=1.0)
    r = Resistor("R1", "1k")
    cap = C("1u")
    c.add(vin, r, cap)
    c.connect(vin.ports[0], n_in)
    c.connect(vin.ports[1], GND)
    c.connect(r.ports[0], n_in)
    c.connect(r.ports[1], n_out)
    c.connect(cap.ports[0], n_out)
    c.connect(cap.ports[1], GND)
    return c


circ = build_rc()
ac = AnalysisSpec("ac", {"sweep_type": "dec", "n": 50, "fstart": 10.0, "fstop": 1e6})
sweep = SweepSpec(variables={"R1": [1e3, 2e3, 5e3]})
job = Job(circuit=circ, analyses=[ac], sweep=sweep, engine="ngspice")

rows = run_and_measure(
    job,
    [
        PhaseMarginSpec(name="pm", numerator="V(out)", denominator="V(vin)"),
        GainBandwidthSpec(name="gbw", numerator="V(out)", denominator="V(vin)"),
        GainMarginSpec(name="gm", numerator="V(out)", denominator="V(vin)"),
    ],
    return_as="python",
)

rows[:3]